In [ ]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 8.9MB/s 
     |████████████████████████████████| 1.1MB 14.8MB/s 
     |████████████████████████████████| 133kB 29.0MB/s 
     |████████████████████████████████| 890kB 23.0MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 7.2MB 33.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=65a5137ffb9ee540c02f4d7dd56e3a3cac0c8b5383ca9ff639ed94725e7bdd42
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.43 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import pandas as pd
text = pd.read_csv('/content/gdrive/MyDrive/7월진짜완료.csv', engine = 'python')
text

,Content_Date
2020-07-01,등교 중지 안 시켜 목숨 걸 학교 가
2020-07-01,다시 보낸 아니 저번 눈 가리 포승줄 묶 역사 재판장 설 많
2020-07-01,검찰 ㅋㅋ ㅋㅋ 진짜 욕 나온다
2020-07-01,절도 보이스 피싱 사기 코로나 장 발장 이 기도 안 참
2020-07-01,재앙 인 아닌
...,...
2020-07-31,코로나 백신 서둘러 나왔
2020-07-31,짱깨 정마 알
2020-07-31,그렇게 더럽 생각 부의금 왜 받 받
2020-07-31,냐 덤비 아프 드러눕 사진 뿌리 이


In [ ]:
model = torch.load('/content/gdrive/MyDrive/긍분중_all_epoch_4_itr.pth')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

100%|██████████| 995526/995526 [00:00<00:00, 2572462.42B/s]


In [ ]:
device = torch.device('cuda')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
text = text.loc[text['Content_Date'].notnull(),:]

In [ ]:
text=text.reset_index()

In [ ]:
text

,index,Content_Date
0,2020-07-01,등교 중지 안 시켜 목숨 걸 학교 가
1,2020-07-01,다시 보낸 아니 저번 눈 가리 포승줄 묶 역사 재판장 설 많
2,2020-07-01,검찰 ㅋㅋ ㅋㅋ 진짜 욕 나온다
3,2020-07-01,절도 보이스 피싱 사기 코로나 장 발장 이 기도 안 참
4,2020-07-01,재앙 인 아닌
...,...,...
100234,2020-07-31,코로나 백신 서둘러 나왔
100235,2020-07-31,짱깨 정마 알
100236,2020-07-31,그렇게 더럽 생각 부의금 왜 받 받
100237,2020-07-31,냐 덤비 아프 드러눕 사진 뿌리 이


In [ ]:
result = pd.DataFrame(columns=['result'])
for i in range(0,100239):
  logits = test_sentences([text['Content_Date'][i]])
  result = result.append(pd.DataFrame([np.argmax(logits)], columns=['result']), ignore_index=True)


# 각 감정이 얼마나 나왔는지 확인하기

In [ ]:
result

,result
0,2
1,2
2,2
3,2
4,2
...,...
455169,2
455170,2
455171,2
455172,0


In [ ]:
n=0
for i in range(0,454898):
  if result['result'][i]==0:
    n=n+1

In [ ]:
n

83835

In [ ]:
result.to_csv('/content/gdrive/MyDrive/tag_result_2.csv', encoding = 'utf-8-sig')

In [ ]:
text['tag']=result

In [ ]:
text

,level_0,index,tag
0,0,응원 꼭 반납,2
1,1,앞 정도 힘들 문재인 정권 이 힘 된다,2
2,2,문재 인 정부 다 세금 대책 아닌 연명 사 정부 개 돼지 눈앞 돈 표 파 무지,2
3,3,코로나 기획 없 기존 회사 사람 내보내,2
4,4,나이 인데 안타깝 학생 피해자 지만 알 좀 깔끔 벌금 자비 치료 끝,2
...,...,...,...
455169,456995,또 미국 증시 조작 발언 ㅋ,2
455170,456996,글 집안 뒹굴 싫,2
455171,456997,강화 함 요즘 가평 양평 남양주시 인파 장난 아닌데 경찰 출동 길레 인원 통제 알 ...,2
455172,456998,그래 정부 믿 마스크 쓰 해서 난리나 여행 독려 쿠폰 ㅋㅋ 음식 할인 쿠폰 ㅋㅋ 주...,0


In [ ]:
text.to_csv('/content/gdrive/MyDrive/찐_real_final_2.csv', encoding = 'utf-8-sig')

In [ ]:
import pandas as pd
result = pd.read_csv('/content/gdrive/MyDrive/tag_우울_result_0.csv', engine = 'python')
result

,Unnamed: 0
0,2
1,2
2,1
3,1
4,2
...,...
454893,1
454894,2
454895,0
454896,1


In [ ]:
text['tag']=result
text

,level_0,index,tag
0,0,ㅋ 인간 극장 이,2
1,1,성 석문 숙 가게 드나들 경찰 대해 직무 유기 고발장 냈 대구 수성 경찰 경찰 가게...,2
2,2,좋 건강 정보 고맙,1
3,3,뺏 정도 라니 전 킬로 였 인간 승리 네요 응 원합 니 당,1
4,4,오른쪽 남자 뺀 사람 인 알,2
...,...,...,...
454893,456995,나라 응원 일본 아베 응원 이대로 쭉 가,1
454894,456996,내려와라 이만큼 했 됐 무능 알,2
454895,456997,중국 온 세계 전쟁 같 고통 겪 코로나 자연 발생 바이러스 같,0
454896,456998,ㅋㅋㅋ 개소리,1


In [ ]:
text.to_csv('/content/gdrive/MyDrive/찐우울_real_final_0.csv', encoding = 'utf-8-sig')